In [272]:
import pandas as pd
import sklearn
import numpy as np

from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier

from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.cluster import KMeans

In [273]:
cf = pd.read_csv("notebooks/county_facts_with_2012_winner.csv")

# get array of fipscodes to later assign as labels by county
fipscodes = cf["fips"].values
winner = cf["winner"].values

In [274]:
# list of attributes that are the most important according to weka
weka_list = ["SEX255214", "RHI125214", "RHI225214","RHI625214", "RHI725214", 
            "RHI825214", "POP815213", "EDU635213", "LFE305213","HSG495213",
            "HSD310213","INC910213","PVY020213","SBO415207", "RTN131207","LND110210", "POP060210"]

In [275]:
# remember the state abbreviations for later (since we need only numeric values)
y = cf["state_abbreviation"].values
cf = cf.drop(['state_abbreviation'], axis=1)

# our table is now only numeric values
cf = cf[weka_list]

# ???
newcf_norm = (cf - cf.mean()) / (cf.max() - cf.min())
newcf_norm = newcf_norm.div(newcf_norm.sum(axis=1), axis=0)

X = cf.as_matrix() # take our dataframe as numpy matrix
kmeans_model = KMeans(n_clusters=4, random_state=1).fit(X) # build model
labels = kmeans_model.labels_ # our cluster labels
clusters = pd.DataFrame(labels, fipscodes) # our reformed dataset, matching the clusters to the fips codes
clusters = clusters.reset_index() 
clusters.columns = ['fips', 'cluster']
clusters # two columns, [fips, cluster]

for i in range(0, 4): # print the size of each cluster
    print len(clusters.loc[clusters['cluster'] == i])

242
1747
56
912


In [294]:
cf["fips"] = fipscodes
cf["cluster"] = clusters['cluster']
cf["winner"] = winner

fi = pd.DataFrame() # create empty dataframe

for i in range(0,4):
    clustered_facts_grouped = cf[cf["cluster"]==i] # groupby cluster
    
#     newcf_norm = (cf - cf.mean()) / (cf.max() - cf.min())
#     newcf_norm = newcf_norm.div(newcf_norm.sum(axis=1), axis=0)

    y = clustered_facts_grouped["winner"]
    clustered_facts_grouped = clustered_facts_grouped.drop(['fips', 'cluster', 'winner'], axis=1)

    X = clustered_facts_grouped.as_matrix()
        
    model = ExtraTreesClassifier() 
    model.fit(X, y) # find which features, in this cluster, were most beneficial to determining the winner

    z = pd.Series(model.feature_importances_)
    fi = fi.append(z, ignore_index=True)
    
fi.columns = weka_list
fi

,SEX255214,RHI125214,RHI225214,RHI625214,RHI725214,RHI825214,POP815213,EDU635213,LFE305213,HSG495213,HSD310213,INC910213,PVY020213,SBO415207,RTN131207,LND110210,POP060210
0,0.069812,0.047176,0.038755,0.081446,0.052123,0.058306,0.057875,0.085359,0.065200,0.039905,0.059446,0.083528,0.058375,0.030336,0.046750,0.069325,0.056282
1,0.045335,0.075424,0.076133,0.073636,0.063798,0.066185,0.054923,0.065842,0.056649,0.056598,0.055054,0.050070,0.053984,0.015311,0.050404,0.078507,0.062146
2,0.045232,0.124081,0.152305,0.058251,0.022482,0.074343,0.058101,0.038295,0.015816,0.009031,0.014093,0.031773,0.107314,0.042946,0.127470,0.054636,0.023830
3,0.050802,0.057159,0.068486,0.055717,0.070758,0.058415,0.060773,0.060328,0.056446,0.067910,0.066747,0.059233,0.055236,0.040985,0.053568,0.063196,0.054242
